In [28]:
import time

import requests
import csv
import os
from typing import List, Dict, Optional

with open("../sec/alpha_vantage.key", "r") as file:
    av_key: str = file.read()

av_base_url: str = 'https://www.alphavantage.co/query?function='
def get_from_av(url: str) -> Dict:
    response = requests.get(url)
    print(response.json())
    return response.json()

def get_from_av_csv(url: str) -> List[Dict]:
    rows = []
    response = requests.get(url)
    if response.status_code == 200:
        # parse the CSV data
        data = response.text
        reader = csv.DictReader(data.splitlines())
        for row in reader:
            rows.append(row)
    else:
        print("Error:", response.status_code)
    return rows

def get_fx_pair(currency_from: str, currency_to: str) -> Dict:
    url: str = f'{av_base_url}CURRENCY_EXCHANGE_RATE&from_currency={currency_from}&to_currency={currency_to}&apikey={av_key}'
    return get_from_av(url)

def get_symbols() -> List[str]:
    url: str = f'{av_base_url}LISTING_STATUS&apikey={av_key}'
    rows = get_from_av_csv(url)
    return list(map(lambda r: r['symbol'], rows))

def get_symbol(symbol: str) -> Dict:
    url: str = f'{av_base_url}TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&apikey={av_key}'
    return get_from_av(url)

def get_symbol_csv(symbol: str) -> List[Dict]:
    url: str = f'{av_base_url}TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&datatype=csv&apikey={av_key}'
    return get_from_av_csv(url)

def load_prices_from_av(symbols: List[str], to_filename:str):
    symbol_column = 'symbol'
    symbol = symbols[0]
    rows = get_symbol_csv(symbol)
    column_names = [symbol_column] + list(rows[0].keys())

    (write_headers, file_mode) = (False, 'a') if os.path.exists(to_filename) else (True, 'x')

    with open(to_filename, file_mode, newline='') as csvfile:
        csvwriter = csv.DictWriter(csvfile, fieldnames = column_names)

        if write_headers:
            csvwriter.writeheader()

        for symbol in symbols:
            rows = get_symbol_csv(symbol)
            for row in rows:
                row[symbol_column] = symbol
                csvwriter.writerow(row)

            time.sleep(1)



In [29]:
symbols = get_symbols()
symbols

['A',
 'AA',
 'AAA',
 'AAAU',
 'AAC',
 'AAC-U',
 'AAC-WS',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AADI',
 'AADR',
 'AAIC',
 'AAIC-P-B',
 'AAIC-P-C',
 'AAIN',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPB',
 'AAPD',
 'AAPL',
 'AAPU',
 'AAT',
 'AAU',
 'AAWW',
 'AAXJ',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEQ',
 'ABEV',
 'ABG',
 'ABGI',
 'ABIO',
 'ABM',
 'ABNB',
 'ABOS',
 'ABR',
 'ABR-P-D',
 'ABR-P-E',
 'ABR-P-F',
 'ABSI',
 'ABST',
 'ABT',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAB',
 'ACABU',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACAQ',
 'ACAQ-U',
 'ACAQ-WS',
 'ACAX',
 'ACAXR',
 'ACAXU',
 'ACAXW',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACDCW',
 'ACDI',
 'ACDI-U',
 'ACEL',
 'ACER',
 'ACES',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACGN',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHV',
 'ACI',
 'ACIO',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'A

In [30]:
# load_prices_from_av(['MSFT'], f'{os.path.dirname(os.getcwd())}\ExcelMinder\ExcelMinder\Data\Files\stock_prices.csv')
load_prices_from_av(symbols, f'{os.path.dirname(os.getcwd())}\ExcelMinder\ExcelMinder\Data\Files\stock_prices.csv')

ValueError: dict contains fields not in fieldnames: '{'

In [2]:
# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# # url = f'https://www.alphavantage.co/query?function=CURRENCY_EXCHANGE_RATE&from_currency=USD&to_currency=JPY&apikey={av_key}'
# # r = requests.get(url)
# # data = r.json()
#
# currency_pairs: List[List[str]] = [['USD', 'GBP'], ['USD', 'EUR'], ['USD', 'JPY'], ['USD', 'CAD'], ['USD', 'CHF'], ['EUR', 'GBP'], ['EUR', 'JPY'], ['EUR', 'CHF'], ['USD', 'BRL'], ['USD', 'CNY'], ['USD', 'INR']]
#
# pairs: Dict[str,Dict] = {}
# # for p in currency_pairs:
# #     val = get_fx_pair(p[0], p[1])
# #     pairs[p[0] + p[1]] = val
#
# print(pairs)

In [5]:
class Security:
    def __init__(self, symbol: str, name: str, exchange: str, assetType: str, ipoDate: Optional[str], delistingDate: Optional[str], status: str):
        self.symbol = symbol
        self.name = name
        self.exchange = exchange
        self.assetType = assetType
        self.ipoDate = ipoDate
        self.delistingDate = delistingDate
        self.status = status

def parse_securities(file_path: str) -> List[Security]:
    securities = []
    with open(file_path, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            securities.append(Security(
                symbol=row['symbol'],
                name=row['name'],
                exchange=row['exchange'],
                assetType=row['assetType'],
                ipoDate=row['ipoDate'],
                delistingDate=row['delistingDate'],
                status=row['status']
            ))
    return securities

In [10]:
import os

print("File location using os.getcwd():", os.path.dirname(os.getcwd()))

print(f"File location using __file__ variable: {os.path.realpath(os.path.dirname(__file__))}")

File location using os.getcwd(): C:\Work\excel-minder_


NameError: name '__file__' is not defined